In [1]:
import pandas as pd
import numpy as np
import json
import re
import os

# RESULTS_DIR = 'Results/2.- Analog Depth Gaugegemini-3-flash-preview'
# GT_DIR = 'Dataset/2_label'
# SUMMARY_OUTPUT = 'Analysed/Analysis_Summary_Depth_flash.xlsx'

# RESULTS_DIR = 'Results/3.- Analog Depth Gauge_without verniergemini-3-flash-preview_naive'
# GT_DIR = 'Dataset/3_label'
# # SUMMARY_OUTPUT = 'Analysed/Analysis_Summary_dial_pro.xlsx'

# SUMMARY_OUTPUT = 'Analysed/Analysis_Summary_Depth_without_vernier_flash_naive.xlsx'


RESULTS_DIR = 'Results/GPT/3.- Analog Depth Gauge_without vernier_gpt5.2Thinking_naive'
GT_DIR = 'Dataset/3_label'
# SUMMARY_OUTPUT = 'Analysed/Analysis_Summary_dial_pro.xlsx'

SUMMARY_OUTPUT = 'Analysed/Analysis_Summary_Depth_without_vernier_gpt5.2_naive.xlsx'

def calculate_metrics(vlm_df, gt_df):
    """Calculate MAE, RMSE and TMS """
    # removing whitespace and unifying letter case.
    gt_df.columns = [str(c).strip().lower() for c in gt_df.columns]

    vlm_df.columns = [str(c).strip().lower() for c in vlm_df.columns]
    
    gt_df = gt_df.sort_values('timestamp_ms')
    vlm_df = vlm_df.sort_values('ts_ms')
    
    # Nearest Neighbor
    merged = pd.merge_asof(
        vlm_df,
        gt_df,
        left_on='ts_ms',
        right_on='timestamp_ms',
        direction='nearest',
        tolerance=200  # maximam 200ms error
    ).dropna(subset=['encoder_verified_reading', 'reading'])

    if len(merged) < 2:
        print('error code 9137')
        return None

    rvlm = merged['reading'].values
    rgt = merged['encoder_verified_reading'].values

    mae = np.mean(np.abs(rvlm - rgt))
    rmse = np.sqrt(np.mean((rvlm - rgt)**2))
    
    diff_vlm = np.sign(np.diff(rvlm))
    diff_gt = np.sign(np.diff(rgt))
    tms = np.mean(diff_vlm == diff_gt)

    return {
        "MAE": mae,
        "RMSE": rmse,
        "TMS": tms,
        "Valid_Samples": len(merged)
    }

def extract_json_from_text(text):
    try:
        clean_text = re.sub(r'```json|```', '', text).strip()
        return json.loads(clean_text)
    except Exception as e:
        print(f"JSON parsing error: {e}")
        return None

summary_data = []

for file_name in os.listdir(RESULTS_DIR):
    if not file_name.endswith('_Raw_Results.xlsx'):
        continue
    
    seq_id = file_name.replace('_Raw_Results.xlsx', '')
    print(f"Analysing: {seq_id}...")

    vlm_raw_path = os.path.join(RESULTS_DIR, file_name)
    vlm_raw_df = pd.read_excel(vlm_raw_path)
    
    raw_response = vlm_raw_df.iloc[0]['raw_model_response']
    vlm_list = extract_json_from_text(raw_response)
    
    if not vlm_list:
        print('error code 3315')
        continue
    vlm_df = pd.DataFrame(vlm_list)

    gt_path = os.path.join(GT_DIR, f"{seq_id}.xlsx")
    
    if not os.path.exists(gt_path):
        print(f"{gt_path} does not exist")
        print('error code 88632')
        continue
        
#     gt_df = pd.read_excel(gt_path)
    gt_df = pd.read_excel(gt_path, sheet_name='Ground_Truth')
    
    metrics = calculate_metrics(vlm_df, gt_df)
    
    if metrics:
        metrics['sequence_id'] = seq_id
        summary_data.append(metrics)


summary_df = pd.DataFrame(summary_data)


metrics_cols = ['MAE', 'RMSE', 'TMS']
stats_mean = summary_df[metrics_cols].mean()
stats_std = summary_df[metrics_cols].std()

table_2_rows = []
for m in metrics_cols:
    table_2_rows.append({
        "Metric": m,
        "Mean ± SD": f"{stats_mean[m]:.4f} ± {stats_std[m]:.4f}"
    })
table_2_df = pd.DataFrame(table_2_rows)


with pd.ExcelWriter(SUMMARY_OUTPUT) as writer:
    table_2_df.to_excel(writer, sheet_name='Summary_Statistics', index=False)
    
    cols_order = ['sequence_id', 'Valid_Samples', 'MAE', 'RMSE', 'TMS']
    summary_df[cols_order].to_excel(writer, sheet_name='Detailed_Metrics', index=False)

print("\n" + "="*30)
print(f"Results saved to: {SUMMARY_OUTPUT}")
print("\n--- Overall Performance ---")
print(table_2_df.to_string(index=False))
print("\n--- Preview of Detailed Data ---")
print(summary_df[cols_order].head())

Analysing: V1_5 divisions_per_second...
Analysing: V2_10 divisions_per_second...

Results saved to: Analysed/Analysis_Summary_Depth_without_vernier_gpt5.2_naive.xlsx

--- Overall Performance ---
Metric         Mean ± SD
   MAE   7.2653 ± 8.0172
  RMSE 11.0277 ± 10.9438
   TMS   0.7984 ± 0.1419

--- Preview of Detailed Data ---
                  sequence_id  Valid_Samples        MAE       RMSE       TMS
0   V1_5 divisions_per_second             80   1.596250   3.289273  0.898734
1  V2_10 divisions_per_second             54  12.934259  18.766115  0.698113


In [1]:
import pandas as pd
import numpy as np
import json
import re
import os

Model_name = 'GPT' # Gemini GPT
TASK_CONFIGS = [
    {
        "results_dir": f'Results/{Model_name}/1.- Analog Dial Gauge_gpt5.2Thinking_naive',
        "gt_dir": 'Dataset/1_label',
        "label": "dial"
    },
    {
        "results_dir": f'Results/{Model_name}/2.- Analog Depth Gauge_gpt5.2Thinking_naive',
        "gt_dir": 'Dataset/2_label',
        "label": "depth"
    },
    {
        "results_dir": f'Results/{Model_name}/3.- Analog Depth Gauge_without vernier_gpt5.2Thinking_naive',
        "gt_dir": 'Dataset/3_label',
        "label": "linear"
    }
]

SUMMARY_OUTPUT = 'Analysed/Combined_Analysis_Summary_gpt5.2_naive.xlsx'


def calculate_metrics(vlm_df, gt_df):

    gt_df.columns = [str(c).strip().lower() for c in gt_df.columns]
    vlm_df.columns = [str(c).strip().lower() for c in vlm_df.columns]
    
    gt_df = gt_df.sort_values('timestamp_ms')
    vlm_df = vlm_df.sort_values('ts_ms')
    
    merged = pd.merge_asof(
        vlm_df,
        gt_df,
        left_on='ts_ms',
        right_on='timestamp_ms',
        direction='nearest',
        tolerance=200
    ).dropna(subset=['encoder_verified_reading', 'reading'])

    if len(merged) < 2:
        return None

    rvlm = merged['reading'].values
    rgt = merged['encoder_verified_reading'].values

    mae = np.mean(np.abs(rvlm - rgt))
    rmse = np.sqrt(np.mean((rvlm - rgt)**2))
    
    diff_vlm = np.sign(np.diff(rvlm))
    diff_gt = np.sign(np.diff(rgt))
    tms = np.mean(diff_vlm == diff_gt)

    return {
        "MAE": mae,
        "RMSE": rmse,
        "TMS": tms,
        "Valid_Samples": len(merged)
    }

def extract_json_from_text(text):
    try:
        clean_text = re.sub(r'```json|```', '', text).strip()
        return json.loads(clean_text)
    except Exception as e:
        print(f"JSON parsing error: {e}")
        return None
    
summary_data = []

for config in TASK_CONFIGS:
    res_dir = config["results_dir"]
    gt_dir = config["gt_dir"]
    tag = config["label"]
    
    print(f"\n>>> Processing: {tag}")
    
    if not os.path.exists(res_dir):
        print(f"result file does not exist: {res_dir}")
        continue

    for file_name in os.listdir(res_dir):
        if not file_name.endswith('_Raw_Results.xlsx'):
            continue
        
        seq_id = file_name.replace('_Raw_Results.xlsx', '')
        vlm_raw_path = os.path.join(res_dir, file_name)
        
        try:

            vlm_raw_df = pd.read_excel(vlm_raw_path)
            raw_response = vlm_raw_df.iloc[0]['raw_model_response']
            vlm_list = extract_json_from_text(raw_response)
            
            if not vlm_list: continue
            vlm_df = pd.DataFrame(vlm_list)


            gt_path = os.path.join(gt_dir, f"{seq_id}.xlsx")
            if not os.path.exists(gt_path):
                print(f"GT file {gt_path} does not exist")
                continue
                
            gt_df = pd.read_excel(gt_path, sheet_name='Ground_Truth')
            

            metrics = calculate_metrics(vlm_df, gt_df)
            
            if metrics:
                metrics['sequence_id'] = seq_id
                metrics['group_tag'] = tag 
                summary_data.append(metrics)
                
        except Exception as e:
            print(f"Error when processing {file_name}: {e}")

summary_df = pd.DataFrame(summary_data)

metrics_cols = ['MAE', 'RMSE', 'TMS']
stats_mean = summary_df[metrics_cols].mean()
stats_std = summary_df[metrics_cols].std()

table_overall = pd.DataFrame([{
    "Metric": m,
    "Combined Mean ± SD": f"{stats_mean[m]:.4f} ± {stats_std[m]:.4f}"
} for m in metrics_cols])


with pd.ExcelWriter(SUMMARY_OUTPUT) as writer:
    table_overall.to_excel(writer, sheet_name='Overall_Statistics', index=False)

    cols_order = ['group_tag', 'sequence_id', 'Valid_Samples', 'MAE', 'RMSE', 'TMS']
    summary_df[cols_order].to_excel(writer, sheet_name='Detailed_Metrics', index=False)

print(f"\nsaved to: {SUMMARY_OUTPUT}")


>>> Processing: dial

>>> Processing: depth

>>> Processing: linear

saved to: Analysed/Combined_Analysis_Summary_gpt5.2_naive.xlsx
